In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

In [2]:
# Create SparkSession
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [3]:
spark

In [4]:
spark.sql(
  """
  CREATE TABLE carro_delta (id INT, placa STRING) USING delta
  """
)

DataFrame[]

In [5]:
spark.sql("select * from carro_delta").show()

+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [6]:
from delta.tables import DeltaTable

carro = DeltaTable.forPath(spark, "./spark-warehouse/carro_delta")

In [7]:
carro.history().show()

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+
|      0|2025-04-22 00:47:...|  NULL|    NULL|CREATE TABLE|{partitionBy -> [...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|              {}|        NULL|Apache-Spark/3.5....|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+--------------------+



In [8]:
spark.sql("INSERT INTO carro_delta VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [9]:
spark.sql("select * from carro_delta").show()

+---+-------+
| id|  placa|
+---+-------+
|  3|ABV1V23|
|  2|RLC5B93|
|  1|XYZ1J34|
+---+-------+



In [10]:
carro.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                           |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1  

In [11]:
spark.sql(
    """
    alter table carro_delta add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [12]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  3|ABV1V23| NULL|  NULL|NULL|
|  2|RLC5B93| NULL|  NULL|NULL|
|  1|XYZ1J34| NULL|  NULL|NULL|
+---+-------+-----+------+----+



In [13]:
spark.sql(
    """
    update carro_delta set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[num_affected_rows: bigint]

In [14]:
spark.sql(
    """
    select * from carro_delta
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  3|ABV1V23|   NULL|   NULL|NULL|
|  2|RLC5B93|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [15]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/carro_delta")

True

In [16]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  1|XYZ1J34|Renault|Sandero|2021|
|  3|ABV1V23|   NULL|   NULL|NULL|
|  2|RLC5B93|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [17]:
spark.sql('update carro_delta set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[num_affected_rows: bigint]

In [18]:
spark.sql('update carro_delta set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[num_affected_rows: bigint]

In [19]:
spark.sql('select * from carro_delta').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  1|XYZ1J34|Renault| Sandero|2021|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  2|RLC5B93|     GM| tracker|2020|
+---+-------+-------+--------+----+



In [20]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [21]:
spark.sql('describe HISTORY carro_delta').show(truncate=False);

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                                                                           

In [22]:
spark.sql(
  """
  CREATE TABLE cliente_delta (id INT, nome STRING, telefone STRING, email STRING) USING delta
  """
)

DataFrame[]

In [23]:
spark.sql('select * from cliente_delta').show()

+---+----+--------+-----+
| id|nome|telefone|email|
+---+----+--------+-----+
+---+----+--------+-----+



In [24]:
spark.sql("INSERT INTO cliente_delta VALUES (1, 'Joshua', '4899201219', 'joshua@gmail.com'), (2, 'Daniela', '4899201230', 'daniela@gmail.com'), "
"(3, 'Emely', '4899201250', 'emely@gmail.com')")

DataFrame[]

In [25]:
spark.sql('select * from cliente_delta').show()

+---+-------+----------+-----------------+
| id|   nome|  telefone|            email|
+---+-------+----------+-----------------+
|  2|Daniela|4899201230|daniela@gmail.com|
|  1| Joshua|4899201219| joshua@gmail.com|
|  3|  Emely|4899201250|  emely@gmail.com|
+---+-------+----------+-----------------+



In [26]:
spark.sql(
  """
  CREATE TABLE vendas_delta (id INT, id_cliente INT, id_carro INT, preco FLOAT, data DATE) USING delta
  """
)

DataFrame[]

In [27]:
spark.sql('select * from vendas_delta').show()

+---+----------+--------+-----+----+
| id|id_cliente|id_carro|preco|data|
+---+----------+--------+-----+----+
+---+----------+--------+-----+----+



In [28]:
spark.sql("INSERT INTO vendas_delta VALUES (1, 1, 1, 50000.00, NOW()), (2, 2, 2, 25000.00, NOW())")

DataFrame[]

In [29]:
spark.sql('select * from vendas_delta').show()

+---+----------+--------+-------+----------+
| id|id_cliente|id_carro|  preco|      data|
+---+----------+--------+-------+----------+
|  2|         2|       2|25000.0|2025-04-22|
|  1|         1|       1|50000.0|2025-04-22|
+---+----------+--------+-------+----------+

